link for outline
<a id='acquire_playlist_data'></a>

# Imports

In [1]:
import pandas as pd
import numpy as np
import requests

from functions import api

%load_ext autoreload
%autoreload 2

# What are we doing?

## Data pull stuff
- Get list of playlist urls for specific user to use in API endpoint
- Get songs off a specific playlist and store as a list
- Look up data for songs in specific playlist and store them

### Get user profile + playlist

#### Format POST requests (details in spotify_scratchpad.ipynb)

In [5]:
import requests

CLIENT_ID = '1c2e33a8bebd41fbbb8a1ecf0e8c4273'
CLIENT_SECRET = '4964098dcc7b41f99c4178e6403645c1'

AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST

auth_response = requests.post(AUTH_URL, {
    'grant_type' : 'client_credentials', 
    'client_id' : CLIENT_ID,
    'client_secret' : CLIENT_SECRET,   
})

# convert response to json
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

#### Set correct headers for POST request

In [6]:
headers = {
    'Authorization' : 'Bearer {token}'.format(token=access_token)
    
}

#### Set base URL of all spotify API endpoints

In [7]:
BASE_URL = 'https://api.spotify.com/v1/'

#### Get profile

In [5]:
user_id = 'bothsidesdoit'

user_profile = requests.get(BASE_URL + 'users/' + user_id + '/playlists', headers=headers)

user_profile = user_profile.json()

#### Get playlist url endpoints for api

In [6]:
playlist_number = len(user_profile['items'])

playlist_urls = [
    user_profile['items'][number]['href']
    for number 
    in range(0, playlist_number)
]

In [7]:
playlist_urls

['https://api.spotify.com/v1/playlists/1oqsGOPQF8TqVtiFdfqkd6',
 'https://api.spotify.com/v1/playlists/2NcJyzjvUlvKQJsScpLOpu',
 'https://api.spotify.com/v1/playlists/1Zd7dmudNrNwpfShtK6ywf',
 'https://api.spotify.com/v1/playlists/1u6zqNx6dIAwvKHedLl0E2',
 'https://api.spotify.com/v1/playlists/5kXBwKZOEVT7PXV6rZuhnV',
 'https://api.spotify.com/v1/playlists/4fcUMfrQr7h3j6UpVndlKq']

#### Get specific playlist from above url

In [8]:
single_playlist_request = requests.get(
    playlist_urls[0],
    headers=headers
)

In [11]:
single_playlist_request = single_playlist_request.json()

{'collaborative': False,
 'description': '',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/1oqsGOPQF8TqVtiFdfqkd6'},
 'followers': {'href': None, 'total': 0},
 'href': 'https://api.spotify.com/v1/playlists/1oqsGOPQF8TqVtiFdfqkd6',
 'id': '1oqsGOPQF8TqVtiFdfqkd6',
 'images': [{'height': 640,
   'url': 'https://mosaic.scdn.co/640/ab67616d0000b2730c577c93959ad4f76aab306aab67616d0000b273170683264052615fd667788dab67616d0000b2735cb41ba5f1e01d3a932b2e24ab67616d0000b273faa00320d0021bd65b8613db',
   'width': 640},
  {'height': 300,
   'url': 'https://mosaic.scdn.co/300/ab67616d0000b2730c577c93959ad4f76aab306aab67616d0000b273170683264052615fd667788dab67616d0000b2735cb41ba5f1e01d3a932b2e24ab67616d0000b273faa00320d0021bd65b8613db',
   'width': 300},
  {'height': 60,
   'url': 'https://mosaic.scdn.co/60/ab67616d0000b2730c577c93959ad4f76aab306aab67616d0000b273170683264052615fd667788dab67616d0000b2735cb41ba5f1e01d3a932b2e24ab67616d0000b273faa00320d0021bd65b8613db',
   'width': 60}]

#### Get list of song ids from the playlist request

In [43]:
single_playlist_ids = \
[
    single_playlist_request
        ['tracks']
        ['items']
        [track]
        ['track']
        ['id']
    for
        track
    in
        range(
            0, 
            len(single_playlist_request['tracks'])
        )
]

#### Get list of spotify features 

In [51]:
spotify_features_url = BASE_URL + 'audio-features/'

single_playlist_spotify_features_data = [
    requests.get(
        spotify_features_url + track_id,
        headers=headers
    )
    .json()
    for
        track_id
    in
        single_playlist_ids
]

#### Got the data!  Turning the above into a single function
#### input playlist url -> ouput data

In [9]:
def get_playlist_response(playlist_url):
    '''
    GET response for given playlist url from spotify api 
    
    input:
        playlist_url 
            - spotify API url for given playlist
            - string
        
    output:
        json of playlist_url RESPONSE from spotify api
    '''
    return requests.get(
        playlist_url,
        headers=headers
    ).json()
    

def parse_playlist_song_ids(playlist_url_response):
    '''
    parse playlist url RESPONSE for unique track IDs of songs in playlist 
    
    input:
        playlist_url_response
            - RESPONSE from spotify API for url for given playlist
            - json
        
    output:
        list of strings of unique track IDs 
    '''
    
    return [
    single_playlist_request
        ['tracks']
        ['items']
        [track]
        ['track']
        ['id']
    for
        track
    in
        range(
            0, 
            len(playlist_url_response['tracks'])
        )
    ]

def get_spotify_features_from_trackid(track_ids):
    '''
    GET response of spotify features for given list of track ids
    
    input:
        list of track ids
            - track id elements of list are strings
        
    output:
        df of spotify features for given track list 
    '''
    BASE_URL = 'https://api.spotify.com/v1/'
    
    spotify_features_url = BASE_URL + 'audio-features/'

    data = [
        requests.get(
            spotify_features_url + track_id,
            headers=headers
        )
        .json()
        for
            track_id
        in
            track_ids
    ]
    
    return pd.DataFrame(data)

def get_spotify_features(playlist_url):
    '''
    gather spotify features for each track in given playlist url
    
    input:
        playlist_url 
            - spotify API url for given playlist
            - string
        
    output:
        df of spotify features for that playlist
    '''
    
    playlist_response = get_playlist_response(
        playlist_url
    )
    
    playlist_track_ids = parse_playlist_song_ids(
        playlist_response
    )
    
    spotify_features = get_spotify_features_from_trackid(
        playlist_track_ids
    )
    
    return spotify_features

    
    

## Test

In [9]:
BASE_URL = 'https://api.spotify.com/v1/'

# TRACK ID from spotify URI obtained from regular spotify web page
track_id = '6y0igZArWVi6Iz0rj35c1Y'

# actual GET request w/ proper header

r = requests.get(BASE_URL + 'audio-features/' + track_id, headers=headers)

r = r.json()

r

{'danceability': 0.54,
 'energy': 0.59,
 'key': 0,
 'loudness': -4.359,
 'mode': 1,
 'speechiness': 0.0528,
 'acousticness': 0.446,
 'instrumentalness': 0,
 'liveness': 0.14,
 'valence': 0.267,
 'tempo': 119.878,
 'type': 'audio_features',
 'id': '6y0igZArWVi6Iz0rj35c1Y',
 'uri': 'spotify:track:6y0igZArWVi6Iz0rj35c1Y',
 'track_href': 'https://api.spotify.com/v1/tracks/6y0igZArWVi6Iz0rj35c1Y',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6y0igZArWVi6Iz0rj35c1Y',
 'duration_ms': 234910,
 'time_signature': 4}

In [14]:
requests.get(
    'https://api.spotify.com/v1/audio-analysis/6y0igZArWVi6Iz0rj35c1Y',
    headers=headers
).json()

{'meta': {'analyzer_version': '4.0.0',
  'platform': 'Linux',
  'detailed_status': 'OK',
  'status_code': 0,
  'timestamp': 1453954087,
  'analysis_time': 10.86589,
  'input_process': 'libvorbisfile L+R 44100->22050'},
 'track': {'num_samples': 5179756,
  'duration': 234.90958,
  'sample_md5': '',
  'offset_seconds': 0,
  'window_seconds': 0,
  'analysis_sample_rate': 22050,
  'analysis_channels': 1,
  'end_of_fade_in': 0.0,
  'start_of_fade_out': 230.31293,
  'loudness': -4.359,
  'tempo': 119.878,
  'tempo_confidence': 0.13,
  'time_signature': 4,
  'time_signature_confidence': 1.0,
  'key': 0,
  'key_confidence': 0.578,
  'mode': 1,
  'mode_confidence': 0.585,
  'codestring': 'eJxdm1mi3SgMRLfiJZgZ9r-xPqfwe0n6o9OBa2MGqVQlkfLW1Vtrb3_eZ589du_vfkbvtvrYY53n1P2s0eqsZz7lrf625uiz0DyjPqPUd9fZntrf91m9n30YsfLUs3t527toNl-nudapqzytj5mPvK2882mz7--b_a1P26v68G6j8YVeDn-cd8zJF_pmpLpne08ZnX5_XJN1MLtnVOa1G-to7z7PHPxaeXGud-1nZlJn7VZoP6s40dXHYBH9WWu5xN7mS5MtqXs_h006fbzl2avx4eJG9d4e58PTu5ZV9vucwVhn71PqZjE

In [15]:
requests.get(
    'https://api.spotify.com/v1/audio-features/6y0igZArWVi6Iz0rj35c1Y',
    headers=headers
).json()

{'danceability': 0.54,
 'energy': 0.59,
 'key': 0,
 'loudness': -4.359,
 'mode': 1,
 'speechiness': 0.0528,
 'acousticness': 0.446,
 'instrumentalness': 0,
 'liveness': 0.14,
 'valence': 0.267,
 'tempo': 119.878,
 'type': 'audio_features',
 'id': '6y0igZArWVi6Iz0rj35c1Y',
 'uri': 'spotify:track:6y0igZArWVi6Iz0rj35c1Y',
 'track_href': 'https://api.spotify.com/v1/tracks/6y0igZArWVi6Iz0rj35c1Y',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6y0igZArWVi6Iz0rj35c1Y',
 'duration_ms': 234910,
 'time_signature': 4}

In [28]:
single_playlist_request['tracks']['items'][12]['track']['href']

'https://api.spotify.com/v1/tracks/77qRetUgTCYUCQ2nSeHIhx'

In [30]:
requests.get(
'https://api.spotify.com/v1/audio-features/77qRetUgTCYUCQ2nSeHIhx',
    headers=headers
).json()

{'danceability': 0.591,
 'energy': 0.695,
 'key': 11,
 'loudness': -4.153,
 'mode': 1,
 'speechiness': 0.0424,
 'acousticness': 0.0945,
 'instrumentalness': 0.000274,
 'liveness': 0.0889,
 'valence': 0.674,
 'tempo': 118.552,
 'type': 'audio_features',
 'id': '77qRetUgTCYUCQ2nSeHIhx',
 'uri': 'spotify:track:77qRetUgTCYUCQ2nSeHIhx',
 'track_href': 'https://api.spotify.com/v1/tracks/77qRetUgTCYUCQ2nSeHIhx',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/77qRetUgTCYUCQ2nSeHIhx',
 'duration_ms': 306240,
 'time_signature': 4}

In [23]:
playlist_url_response = api.get_playlist_response('https://api.spotify.com/v1/playlists/1oqsGOPQF8TqVtiFdfqkd6')

In [25]:
playlist_url_response['tracks']

{'href': 'https://api.spotify.com/v1/playlists/1oqsGOPQF8TqVtiFdfqkd6/tracks?offset=0&limit=100',
 'items': [{'added_at': '2020-06-23T21:31:28Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/bothsidesdoit'},
    'href': 'https://api.spotify.com/v1/users/bothsidesdoit',
    'id': 'bothsidesdoit',
    'type': 'user',
    'uri': 'spotify:user:bothsidesdoit'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/72PnPUc1qv9UjRPaGVZ1jq'},
       'href': 'https://api.spotify.com/v1/artists/72PnPUc1qv9UjRPaGVZ1jq',
       'id': '72PnPUc1qv9UjRPaGVZ1jq',
       'name': 'The Handsome Family',
       'type': 'artist',
       'uri': 'spotify:artist:72PnPUc1qv9UjRPaGVZ1jq'}],
     'available_markets': [],
     'external_urls': {'spotify': 'https://open.spotify.com/album/3u4rZDOy1c7N0otfHS0qTj'},
     'href': 'https://api.spotify.com/v1/albums/3u4rZ

In [26]:
api.parse_playlist_song_ids(playlist_url_response)

['4B4xOuj22g8WAMvTzPV7Bq',
 '6qRmb4uhIUaWE5LySgSfQS',
 '3uroe3JBpsFOOHgOAjyzhG',
 '7f5C7QT51TYQJ7SfOxx0Rm',
 '6K5GAopLZzzg0ZX8k5GWwh',
 '2ShuJDejHCsnULbn9IYitk',
 '0Qty2GKY7Ce1eIsR44QEnu']